In [4]:
import lightgbm

import numpy as np
import pandas as pd

qrels é o arquivo que carrega a relevância que uma certa entidade, ou documento, tem para uma consulta. No nosso caso 0: nada relevante, 1: pouco relevante e 2: bastante relevante.

In [7]:
qrels = pd.read_csv('../data/train_qrels.csv')
qrels.head()

,QueryId,EntityId,Relevance
0,9,367937,1
1,9,429992,2
2,9,513435,1
3,9,571751,2
4,9,582040,2


Features seria o score que cada um dos modelos que a gente implementar da pra um par (Consulta, Documento). Então por exemplo, uma feature pode ser BM25, outra TFIDF, outra Dense Retrieval e por aí vai. O Learning to Rank (se tudo der certo) vai aprender a dar relevancia para cada uma dessas features extraindo um pouquinho (e talvez o melhor) de cada uma.

Agora parando pra pensar esse merge vai ser bem mais complicado do q eu tinha imaginado de primeira, pq tem elements q não vão estar presentes em alguns casos, nesses casos acho que eu vou só por 0 nessa célula. Se eu for retornar os 100 melhores de cada feature e eu usar 5 features, pode acontecer de ter 500 amostras por consulta (exagero da minha parte), vou ter que analisar isso com cuidado.

In [ ]:
features_paths = ["../results/bm25_output/", "../results/tf_idf_output/"]

features_df
for feature in features_paths:
    current_feature = pd.read_csv(feature)
    features_df = pd.concat([feature_df, current_feature], axis=1, ignore_index=True)

features_df.head()

a gente vai juntar nossas features com a relevancia do gabarito (na pratica só pra ter um negócio bonito pra ver...)

In [ ]:
ltr_df = pd.merge(qrels, features_df,  how='left', left_on=['QueryId','EntityId'])
ltr_df.head()

É melhor eu escrever isso se não vou esquecer...
Primeiro eu obtenho os ids de todas as consultas de teste dando um ```unique()``` na coluna de ids e armazendo em ```querie_ids```. Em seguida acho a quantidade de ids únicos e armazeno em ```amount_of_queries```. <br>

Minha proporção treino validação vai ser 80/20, então o tanto de consultas no treino vai ser ```train_size = 0.8 * amount_of_queries``` e o tanto de consultas no val  ```0.2 * amount_of_queries``` (como tenho quantidades de sample para consulta variaveis o buraco é mais embaixo). 

Agora vou achar a consulta que vai ser o ponto de "corte" fazendo ```cut = queries_ids[train_size]```. Assim para achar o df de treino basta selecionar todas as rows cujo ```QueryId > cut```, o restante será o teste.


Obs: Não existe amostra de uma mesma consulta em treino e teste. Fiz isso propositalmente pois não tinha certeza se isso se enquadrava em data leakage.

In [21]:

queries_ids = qrels['QueryId'].unique()
amount_of_queries = len(querie_ids)

train_size = int(0.8 * amount_of_queries)
val_size = amount_of_queries - train_size

cut = querie_ids[train_size]
train_df = ltr_df[ltr_df['QueryId'] <= cut]
val_df = ltr_df[ltr_df['QueryId'] > cut]

test_df.head()

NameError: name 'querie_ids' is not defined

Seguinte, o formato para o lgbm é confuso, mas na pratica x_train é as features, y_train vai ser a relevancia e group vai dividir todas as amostras pela query a qual essa se refere. Então se a primeira query tem 10 samples, a segunda 12, a terceira 14, e a quinta 16, ```group = [10, 12, 14, 16]```. Veja bem, nesse exemplo hipotetico ```x_train``` possui 10 linhas referentes a primeira query, 12 referente a segunda... e a mesma coisa para ```y_val```!

In [ ]:
qids_train = train_df.groupby("QueryId")["QueryId"].count().to_numpy()
X_train = train_df.drop(["QueryId", "Relevance"], axis=1)
y_train = train_df["Relevance"]

In [ ]:
qids_validation = val_df.groupby("QueryId")["QueryId"].count().to_numpy()
X_validation = val_df.drop(["QueryId", "Relevance"], axis=1)
y_validation = val_df["Relevance"]

Agora é instanciar o LightGMB, tem um monte de hiperparametro e _acho_ que esse [link](https://neptune.ai/blog/lightgbm-parameters-guide) aqui tem uma explicação que vai ser ótima pro futuro...

In [ ]:
model = lightgbm.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
)

In [ ]:
model.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_validation, y_validation)],
    eval_group=[qids_validation],
    eval_at=10,
    verbose=10,
)

Agora é que são elas, até onde eu entendo eu vou ter q gerar as features todas para o teste (arquivo separado) e mergear ela (não parece trivial), e aí passar pra função abaixo (para cada consulta (???)). Cada row vai ser uma entidade né, então vou ter um monte de score para cada entidade, aí é ordenar pivotando para ter os ids ordenados tb e retornar esses ids.

In [ ]:
test_pred = gbm.predict(X_test)
X_test["predicted_ranking"] = test_pred
X_test.sort_values("predicted_ranking", ascending=False)

### Referências

- https://tamaracucumides.medium.com/learning-to-rank-with-lightgbm-code-example-in-python-843bd7b44574
- https://towardsdatascience.com/how-to-evaluate-learning-to-rank-models-d12cadb99d47
- https://towardsdatascience.com/how-to-implement-learning-to-rank-model-using-python-569cd9c49b08
- https://stackoverflow.com/questions/64294962/how-to-implement-learning-to-rank-using-lightgbm
- https://www.kaggle.com/code/bturan19/lightgbm-ranker-introduction/notebook
- https://stackoverflow.com/questions/62555987/lightgbm-ranking-example/67621253#67621253
- https://neptune.ai/blog/lightgbm-parameters-guide
- https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRanker.html#
- https://github.com/uni-assignments/research-challenge-2/blob/master/src/ltr.py